# 讀取 DRG 3.4版 1062項 權重表 

In [1]:
import pandas as pd
import numpy as np

df0 = pd.read_excel('NHI/29701_1_105年3.4版1,062項DRG105年權重表(草案).xlsx',header = 11 )

cost1 = df0[:-1] 

cost1.loc[:,'DRG'] = cost1.loc[:,'DRG'].replace(r'^(\d\d\d)$','00\\1' , regex=True)
cost1.loc[:,'DRG'] = cost1.loc[:,'DRG'].replace(r'^(\d\d\d\d)$','0\\1' , regex=True)
cost1.loc[:,'DRG'] = cost1.loc[:,'DRG'].replace(r'^(\d+)','DRG\\1' , regex=True)

print len(cost1)
cost1[10:15]

1062


/Library/Python/2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,MDC,流水號,DRG,RW,個案數<20註記,邏輯註記,幾何平均住院日,下限臨界點,上限臨界點
10,PRE,11,DRG48201,2.7254,NaN,NaN,12,24614,241318
11,PRE,12,DRG48202,2.6825,NaN,NaN,10,28165,272443
12,PRE,13,DRG00512,12.6895,＊,NaN,16,462335,570755
13,1,1,DRG00201,3.9266,NaN,NaN,14,55940,312454
14,1,2,DRG00202,2.7115,NaN,NaN,9,49323,227696


In [79]:
cost1 [ cost1.DRG.str.contains('N') ].head(2)

,MDC,流水號,DRG,RW,個案數<20註記,邏輯註記,幾何平均住院日,下限臨界點,上限臨界點
895,15,1,N01,0.6755,NaN,NaN,4,5923,59471
896,15,2,N02,0.0000,＊,NaN,0,0,0


# 讀取DRG 3.4版 分類表

In [66]:
sheet = pd.read_excel('NHI/29705_1_Tw-DRG3.4版分類表_ICD-10-CMPCS_1041208.xlsx',sheetname=[x for x in range(5,28)], index_col=None)

## 把每個 sheet 讀進到 dataframe

In [76]:
step3.DRG.unique().tolist()[0]

u'DRG48602'

In [78]:

for tab in range(5,28):
    DRG = sheet[tab]
    DRG = DRG.reset_index()
    print DRG.head(1)
    
    step1 = DRG [ ~DRG[DRG.columns[4]].str.contains('-').fillna(False) & pd.notnull(DRG[DRG.columns[0]])  & pd.isnull(DRG[DRG.columns[1]]) & pd.notnull(DRG[DRG.columns[2]]) & pd.notnull(DRG[DRG.columns[3]]) ]
    step1 = step1[['index',DRG.columns[2], DRG.columns[3]]]
    step1.columns = ['DRG','CM-PCS','en']
    step1.loc[:,'DRG'] = step1.loc[:,'DRG'].str.replace(u'\u3001',',' )
    
    for x in step1.DRG.unique():
        y = x.split(',')
        #print y
        for z in y:
            #print z
            step3 = step1 [ step1.DRG.str.contains(x) ]
            step3.DRG = z
            step3.loc[:,'DRG'] = step3.loc[:,'DRG'].replace(r'^(\d\d\d)$','00\\1' , regex=True)
            step3.loc[:,'DRG'] = step3.loc[:,'DRG'].replace(r'^(\d\d\d\d)$','0\\1' , regex=True)
            step3.loc[:,'DRG'] = step3.loc[:,'DRG'].replace(r'^(\d+)','DRG\\1' , regex=True)
            #print step3.head(1)
            step3.to_csv('DRG3.4/%s.csv' %(step3.DRG.unique().tolist()[0]), index=False, encoding='utf-8')
            
    step2 = DRG [ DRG[ DRG.columns[1] ].str.contains('DRG') & pd.notnull(DRG[DRG.columns[2]]) & pd.isnull(DRG[DRG.columns[3]]) & pd.isnull(DRG[DRG.columns[4]])]
    step2[step2.columns[[1,2]]].to_csv('DRG3.4/tab_%s.csv' %(tab),index=False, encoding='utf-8',sep='\t',header=False)
    

  index 附表7.1        Tw-DRGs分類表 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4
0   NaN                     NaN        NaN        NaN        NaN        NaN
  index MDC1                                    神經系統之疾病與疾患 Unnamed: 2  \
0   NaN  NaN  DISEASES AND DISORDERS OF THE NERVOUS SYSTEM        NaN   

  Unnamed: 3 異動註記  
0        NaN  NaN  
  index MDC2                            眼之疾病與疾患 Unnamed: 2 Unnamed: 3 異動註記
0   NaN  NaN  DISEASES AND DISORDERS OF THE EYE        NaN        NaN  NaN
  index MDC3                                       耳鼻喉及口腔之疾病與疾患 Unnamed: 2  \
0   NaN  NaN  DISEASES AND DISORDERS OF THE EAR, NOSE, MOUTH...        NaN   

  Unnamed: 3 異動註記  
0        NaN  NaN  
  index MDC4                                       呼吸系統之疾病與疾患 Unnamed: 2  \
0   NaN  NaN  DISEASES AND DISORDERS OF THE RESPIRATORY SYSTE        NaN   

  Unnamed: 3 異動註記  
0        NaN  NaN  
  index MDC5                                       循環系統之疾病與疾患 Unnamed: 2  \
0   NaN  NaN  DISEASES AND DISORDERS OF THE C

In [ ]:

#step1 = DRG [ ~ pd.notnull(DRG[u'異動註記']) & pd.isnull(DRG.MDC1) & pd.notnull(DRG['Unnamed: 2'])]
#step1 = step1[['index',u'神經系統之疾病與疾患', 'Unnamed: 2']]
#step1.columns = ['DRG','CM-PCS','en']
#step1.loc[:,'DRG'] = step1.loc[:,'DRG'].str.replace(u'\u3001',',' )
#step1.groupby('DRG')['CM-PCS'].size()
#A = step1.groupby('DRG')['CM-PCS'].apply(lambda x: x.tolist())
#step2 = pd.DataFrame(A)
#step2 = step2.reset_index()
#step2.DRG.str.split(',')

#step2.head(3)

In [ ]:
#MDC = pd.DataFrame()
#for x in step1.DRG.unique():
#    y = x.split(',')
#    #print y
#    for z in y:
#        #print z
#        step3 = step1 [ step1.DRG.str.contains(x) ]
#        step3.DRG = z
#        MDC = pd.concat([step3,MDC])#

#MDC.loc[:,'DRG'] = MDC.loc[:,'DRG'].replace(r'^(\d\d\d)$','00\\1' , regex=True)
#MDC.loc[:,'DRG'] = MDC.loc[:,'DRG'].replace(r'^(\d\d\d\d)$','0\\1' , regex=True)
#MDC.loc[:,'DRG'] = MDC.loc[:,'DRG'].replace(r'^(\d+)','DRG\\1' , regex=True)
#MDC